In [2]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
# Importando o módulo de Regressão Linear do scikit-learn
from sklearn.linear_model import LinearRegression
from Análise_de_Correlação import analiseCorr
from Normalização import normalizaDf
df= pd.read_excel("input_acoes_NotasFundamentei_20200517.xlsx")
abas= pd.ExcelFile("input_planilhaCanalHolder_20200211.xlsx")
MA=abas.parse('Múltiplos Ações')
FA=abas.parse('Fundamentos Ações')

In [3]:
#Join das tabelas "Fundamentos ações" e "NotasCanalFundamentei" e um merge entre as duas primeiras abas do arquivo input_planilhaCanalHolder_20200211.xlsx
MA=MA.join(df.set_index("Código de Neg."),on="Código de Neg.")
uniaoAbas = pd.merge(MA, FA, on="Nome", how='outer', suffixes=('', '_y'))
uniaoAbas = uniaoAbas.drop(list(uniaoAbas.filter(regex='_y$')), axis=1)
#Remoção de uma coluna inútil
uniaoAbas.drop("Unnamed: 0",axis=1,inplace=True)

In [7]:
#Removando uma coluna de notas que nao foi utilizada, pois tem dados faltando
uniaoAbas.drop("NotaFundamentei20200201",axis=1,inplace=True)

In [45]:
#Selecionando todas as empresas que possuem notas e estão com seus dados completos
empresasComNotas=uniaoAbas.replace("-",np.nan)
empresasComNotas.dropna(inplace=True)

In [47]:
#Selecionando todas as empresas que não possuem notas e estão com seus dados completos
empresasSemNotas=uniaoAbas[uniaoAbas["NotaFundamentei"].isnull()]
empresasSemNotas.drop("NotaFundamentei",axis=1,inplace=True)
empresasSemNotas.replace("-",np.nan,inplace=True)
empresasSemNotas.dropna(inplace=True)

c:\users\danie\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
c:\users\danie\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:4166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-47-c8476afd1244>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empresasSemNotas.dropna

In [48]:
#Normalizando os dataFrames das empresas com e sem notas, respectivamente
dfCNota=normalizaDf(empresasComNotas)
dfSNota=normalizaDf(empresasSemNotas)

In [59]:
#X=Variáveis dependentes das empresas com nota, X1=Variaveis dependentes das empresas sem nota, Y=Variável
#independente, as notas das empresas
X=dfCNota.drop("NotaFundamentei",axis=1)
X1=dfSNota
Y=dfCNota["NotaFundamentei"]

In [60]:
# Criando o objeto de regressão linear
regr = LinearRegression()

In [61]:
# Treinando o modelo usando os valores das empresas com dados e os resultados dessas empresas 
regr.fit(X, Y)

LinearRegression()

In [71]:
#Criando uma coluna com as notas previstas para as empresas com notas
empresasComNotas["Nota Prevista"]=regr.predict(X)

In [72]:
#Criando uma coluna com as notas previstas para as empresas sem notas
empresasSemNotas["Nota Prevista"]=regr.predict(X1)

<ipython-input-72-865ff82567c9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empresasSemNotas["Nota Prevista"]=regr.predict(X1)


In [79]:
#concatenando os dataFrames, OBS: ambos possuem as mesmas colunas, exceto pela coluna de "NotaFundamente",
# na qual,somento o dataFrame "EmpresasComNotas" possue.
empresas=pd.concat([empresasComNotas, empresasSemNotas], sort=False)

In [84]:
#Normalizando a nota prevista de todas as empresas
empresas["Nota Prevista"]=((empresas["Nota Prevista"]-empresas["Nota Prevista"].min())/(empresas["Nota Prevista"].max()-empresas["Nota Prevista"].min()))*10

In [85]:
#Arredondando a nota para uma casa decimal apenas
empresas["Nota Prevista"]=empresas["Nota Prevista"].round(1)

In [104]:
#Ordenando as empresas com base na sua nota prevista, as empresas contendo "NaN" na coluna "NotaFundamente"
#são aquelas que não tiveram suas notas fornecidas, possuindo apenas a nota prevista, gerada por meio de 
# um regressor
empresas.sort_values("Nota Prevista",ascending=False)[["Nome","Código de Neg.","Nota Prevista","NotaFundamentei"]]

,Nome,Código de Neg.,Nota Prevista,NotaFundamentei
213,Klabin S/A,KLBN11,10.0,NaN
211,Klabin S/A,KLBN11,10.0,NaN
212,Klabin S/A,KLBN11,10.0,NaN
334,Taesa,TAEE11,9.7,NaN
226,Lojas Americ,LAME4,9.7,NaN
...,...,...,...,...
84,CCR SA,CCRO3,0.2,2.4
63,BR Malls Par,BRML3,0.2,2.6
170,Grupo Natura,NTCO3,0.2,2.4
193,Itausa,ITSA4,0.0,NaN


In [109]:
empresass=empresas.drop_duplicates(subset="Nome")

In [114]:
#Informando o resultado acima porém, sempre empresas duplicadas
empresass.sort_values("Nota Prevista",ascending=False)[["Nome","Código de Neg.","Nota Prevista","NotaFundamentei"]].head(60)

,Nome,Código de Neg.,Nota Prevista,NotaFundamentei
211,Klabin S/A,KLBN11,10.0,NaN
332,Taesa,TAEE11,9.7,NaN
225,Lojas Americ,LAME4,9.7,NaN
238,Marfrig,MRFG3,9.6,NaN
259,P.Acucar-Cbd,PCAR4,9.5,NaN
199,JBS,JBSS3,9.3,NaN
364,Ultrapar,UGPA3,9.0,NaN
124,Ecorodovias,ECOR3,8.7,NaN
156,Gerdau,GGBR4,8.3,NaN
94,Cemig,CMIG4,8.3,NaN


In [101]:
#Apenas uma ferramente de pesquisa, avaliando se a nota prevista ordenou as empresas que ja possuiam 
# notas anteriormente de forma correta.
df[df["Código de Neg."]=="WEGE3"]

,Código de Neg.,NotaFundamentei,NotaFundamentei20200201
0,WEGE3,5.0,4.9
